In [ ]:
import torch
import torch.onnx
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
# Загрузка предобученной модели GRU
model = torch.nn.GRU(10, 20, 2)

In [ ]:
dummy_input = torch.randn(5, 3, 10)

In [ ]:
h0 = torch.randn(2, 3, 20)

In [ ]:
output, hn = model(dummy_input, h0)

In [ ]:
print("Результат запуска модели GRU в PyTorch:", output)

Результат запуска модели GRU в PyTorch: tensor([[[ 1.8866e-01, -6.6398e-01,  3.5875e-01,  6.3592e-01, -1.6016e-02,
          -1.0286e-01, -1.1579e-01,  5.3281e-01,  2.3603e-01, -4.2041e-01,
           3.9584e-01,  1.2131e+00, -3.4559e-01,  7.5287e-01, -5.5224e-01,
           3.3739e-01,  1.9625e-01,  1.8087e-01, -6.1101e-01, -5.5379e-01],
         [-4.4994e-01, -9.8108e-01,  1.5562e+00, -6.8241e-01, -5.0831e-01,
          -2.5018e-01, -5.4697e-01, -4.6676e-01, -6.3951e-02,  9.1709e-01,
           1.4041e-01, -5.1273e-01, -3.6976e-01,  7.7186e-02, -1.0161e+00,
          -3.2855e-02, -3.5196e-01,  3.7244e-01, -5.3962e-01, -1.1579e-01],
         [ 5.6727e-01,  7.1473e-01, -6.9649e-01, -2.1437e-01, -9.4801e-01,
          -7.6461e-02, -3.3181e-01, -5.1095e-01,  3.0879e-01,  8.2313e-01,
           4.5968e-02,  1.7292e-01,  1.1189e+00, -1.3647e-01,  5.4413e-01,
           1.1749e-01, -2.6013e-01,  7.0153e-01,  7.5298e-01,  6.2988e-01]],

        [[-1.6473e-01, -4.5812e-01,  3.1957e-01,  3.959

In [ ]:
# Сохранение модели в формате ONNX
torch.onnx.export(model, dummy_input, "gru.onnx", verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [ ]:
# Загрузка модели из файла ONNX
import onnx
import onnxruntime
onnx_model = onnx.load("gru.onnx")
ort_session = onnxruntime.InferenceSession("gru.onnx")

In [ ]:
# Запуск модели из файла ONNX
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели GRU в ONNX:", ort_outs)

Результат запуска модели GRU в ONNX: [array([[[-5.13831601e-02, -8.04906785e-02,  1.34938955e-01,
         -8.79507139e-02, -1.28573149e-01, -4.99530733e-02,
          2.57793982e-02,  1.19425124e-02,  3.42397764e-02,
          2.49518231e-02,  1.49287939e-01,  7.76931271e-02,
         -3.10726538e-02,  7.72229135e-02,  4.87266928e-02,
         -9.01821628e-02,  6.66254479e-03,  2.13070270e-02,
          3.93829457e-02,  4.58321609e-02],
        [ 2.52850745e-02, -5.28012998e-02,  1.47769377e-01,
         -1.98535658e-02, -5.22501357e-02, -3.29307077e-04,
          3.20549193e-03,  6.48132106e-03,  1.06334798e-01,
          5.48448861e-02,  1.32561758e-01,  8.64244401e-02,
         -2.29135398e-02,  1.37331799e-01,  1.28460571e-01,
         -6.55954108e-02,  1.03387035e-01,  7.63445348e-02,
          8.16536974e-03,  4.19627242e-02],
        [ 4.32912149e-02, -9.21377316e-02,  1.25439554e-01,
         -7.86349103e-02, -1.91919267e-01, -1.13616593e-01,
          3.68109532e-02, -2.93387

In [ ]:
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("gru.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.8 MB/s eta 0:00:00


NotImplementedError: ignored